Intro

In [12]:
import pandas as pd
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

In [ ]:
fasta_file = "../data_prep/omic_data/H3N2_HA_Merged_Complete.fasta"
output_fasta = "../data_prep/omic_data/H3N2_HA_Merged_Complete_proteins.fasta"

protein_records = []

for record in SeqIO.parse(fasta_file, "fasta"):
    dna_seq = record.seq

    protein_seq = Seq(str(dna_seq)).translate(to_stop=True)

    protein_record = SeqRecord(
        protein_seq,
        id=record.id,
        description= record.description + ", translated_protein"
    )

    protein_records.append(protein_record)

SeqIO.write(protein_records, output_fasta, "fasta")

print(f"Tradução concluída: {len(protein_records)} proteínas guardadas")

Tradução concluída: 240 proteínas guardadas


c:\Users\Alexandre\AppData\Local\Programs\Python\Python312\Lib\site-packages\Bio\Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(
